In [1]:
import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.algorithms.optimizers import SPSA
import numpy as np

from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.providers.aer import AerSimulator


In [2]:
def PoolingCirc(qc, weights, wires):
    qc.cry(weights[0], wires[0], wires[1])
    qc.x(wires[0])
    qc.cry(weights[1], wires[0], wires[1])
    qc.x(wires[0])

def ConvCirc1(qc, weights, wires):
    qc.ry(weights[0], wires[0])
    qc.ry(weights[1], wires[1])
    qc.cnot(wires[0], wires[1])

inputs = ['+X', '-X', '+Y', '-Y', '+Z', '-Z']

quantum_instance = QuantumInstance(backend=AerSimulator(), shots=1024, optimization_level=0)

In [3]:
def error_channel_circuit(qc, params, data, p):

    if data not in inputs:
        raise ValueError('Invalid Input')

    # Data encoding
    if data == '+X':
        qc.h(0)
    elif data == '-X':
        qc.x(0)
        qc.h(0)
    elif data == '+Y':
        qc.rx(-np.pi/2, 0)
    elif data == '-Y':
        qc.rx(np.pi/2, 0)
    elif data == '-Z':
        qc.x(0)

    # Shor Encoding
    qc.cnot(0,1)
    qc.cnot(0,2)

    #Error Generating
    dperror = qiskit.providers.aer.noise.depolarizing_error(param = p, num_qubits = 3, standard_gates=None)
    dperror = dperror.to_instruction()
    qc = qc.compose(dperror)

    #QCNN
    ConvCirc1(qc, params[0:2], wires = [0, 1])  
    ConvCirc1(qc, params[2:4], wires = [2, 3])    
    ConvCirc1(qc, params[4:6], wires = [1, 2])
    ConvCirc1(qc, params[6:8], wires = [3, 0])
    qc.barrier()
    PoolingCirc(qc, params[8:10], wires = [0, 1])
    PoolingCirc(qc, params[10:12], wires = [2, 3])
    qc.barrier()
    ConvCirc1(qc, params[12:14], wires = [1, 3])
    qc.barrier()
    PoolingCirc(qc, params[14:16], wires = [1, 3])
    qc.barrier()

    #adjoint
    if data == '+X':
        qc.h(3)
    elif data == '-X':
        qc.x(3)
        qc.h(3)
    elif data == '+Y':
        qc.rx(+np.pi/2, 3)
    elif data == '-Y':
        qc.rx(-np.pi/2, 3)
    elif data == '-Z':
        qc.x(3)

    qc.measure(3, 0)

    return qc

In [8]:
quantum_instance = QuantumInstance(backend=AerSimulator(), shots=1024, optimization_level=0)

def error_channel_cost(params, p):
    ret = 0
    for input in inputs:
        qc = QuantumCircuit(4, 1)
        qc = error_channel_circuit(qc, params, input, p)
        ret_dict = quantum_instance.execute(qc).get_counts()
        ret += -ret_dict['0']/sum(list(ret_dict.values()))
    return ret/len(inputs)



def error_channel_train(initparams, p):
    cost_list = []
    def callback(*args):
        cost_list.append(args[2])
        #print('{} Iter | Cost {}'.format(int(args[0]/3), args[2]))

    def wrap_error_channel(params):
        return error_channel_cost(params, p)

    optim = SPSA(maxiter = 500, blocking = True, callback=callback)

    result = optim.minimize(wrap_error_channel, initparams)
    return result.x, cost_list, min(cost_list)

In [9]:
def shor_error_channel_circuit(qc, data, p):

    if data not in inputs:
        raise ValueError('Invalid Input')

    # Data encoding
    if data == '+X':
        qc.h(0)
    elif data == '-X':
        qc.x(0)
        qc.h(0)
    elif data == '+Y':
        qc.rx(-np.pi/2, 0)
    elif data == '-Y':
        qc.rx(np.pi/2, 0)
    elif data == '-Z':
        qc.x(0)

    # Shor Encoding
    qc.cnot(0,1)
    qc.cnot(0,2)

    #Error Generating
    dperror = qiskit.providers.aer.noise.depolarizing_error(param = p, num_qubits = 3, standard_gates=None)
    dperror = dperror.to_instruction()
    qc = qc.compose(dperror)

    #Shor Decoder
    qc.cnot(0, 1)
    qc.cnot(0, 2)
    qc.toffoli(2, 1, 0)

    #adjoint
    if data == '+X':
        qc.h(0)
    elif data == '-X':
        qc.x(0)
        qc.h(0)
    elif data == '+Y':
        qc.rx(+np.pi/2, 0)
    elif data == '-Y':
        qc.rx(-np.pi/2, 0)
    elif data == '-Z':
        qc.x(0)

    qc.measure(0, 0)

    return qc

In [10]:
quantum_instance = QuantumInstance(backend=AerSimulator(), shots=1024, optimization_level=0)

def shor_cost(p):
    ret = 0
    for input in inputs:
        qc = QuantumCircuit(4, 1)
        qc = shor_error_channel_circuit(qc, input, p = p)
        ret_dict = quantum_instance.execute(qc).get_counts()
        ret += -ret_dict['0']/sum(list(ret_dict.values()))
    return ret/len(inputs)

In [11]:
for p in [0.1, 0.01, 0.001, 0.0001]:
    print(p)
    params, costlist, mincost = error_channel_train(np.random.rand(16), p)
    shorcost = shor_cost(p)
    print('Shor: {} | QCNN: {}'.format(shorcost, mincost))

0.1
Shor: -0.7981770833333334 | QCNN: -0.76318359375
0.01
Shor: -0.830078125 | QCNN: -0.74853515625
0.001
Shor: -0.8326822916666666 | QCNN: -0.8359375
0.0001
